In [3]:
!echo '{"username":"santhoshkolloju6693","key":"f2a1d402f1c35e8a0e74d6373eeec602"}' > ~/.kaggle/kaggle.json
!kaggle competitions download -c tensorflow2-question-answering

  0% 0.00/4.78M [00:00<?, ?B/s]
100% 4.78M/4.78M [00:00<00:00, 79.1MB/s]
  0% 0.00/18.2k [00:00<?, ?B/s]
100% 18.2k/18.2k [00:00<00:00, 57.8MB/s]
100% 4.46G/4.46G [01:35<00:00, 40.7MB/s]
100% 4.46G/4.46G [01:35<00:00, 50.0MB/s]


In [4]:
!unzip simplified-nq-train.jsonl.zip

Archive:  simplified-nq-train.jsonl.zip
  inflating: simplified-nq-train.jsonl  


In [0]:
import json
with open("simplified-nq-train.jsonl","r") as f:
  data = json.load(f)

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import json


In [0]:
with open('/content/drive/My Drive/train.json','r') as f:
  squad_train = json.load(f)

In [0]:
combine_cq =[]

In [0]:
for i in range(len(squad_train['data'])):
  for j in range(len(squad_train['data'][i]['paragraphs'])):
    for k in range(len(squad_train['data'][i]['paragraphs'][j]['qas'])):
      combine_cq.append({
          "question":squad_train['data'][i]['paragraphs'][j]['qas'][k]['question'],
          "context":squad_train['data'][i]['paragraphs'][j]['context']
          
          
      })
      

In [0]:
combine_cq = combine_cq[240000:]

In [0]:
combine_cq[-1008]

{'context': "Construction work is carried out on several lines of the new Wuhan Metropolitan Area Intercity Railway, which will eventually connect Wuhan's three main rail terminals with several stations throughout the city's outer areas and farther suburbs, as well as with the nearby cities of Xianning, Huangshi, Huanggang, and Xiaogan. The first line of the system, the one to Xianning, opened for passenger operations at the end of 2013.",
 'question': "what is the name of the railway that will connect wuhan 's three main rail terminals ?"}

In [0]:
!git clone https://github.com/martiansideofthemoon/squash-generation.git
!cp -r squash-generation/pytorch-pretrained-BERT/pytorch_pretrained_bert .
!rm -r train_data eval_data
!mkdir train_data
!mkdir eval_data
for i in range(len(combine_cq)):
  file_name = "train_data/example"+str(i)+".txt"
  with open(file_name,'w') as f:
    f.write(str(combine_cq[i]['context'].strip().replace("|||",""))+"|||"+str(combine_cq[i]['question'].strip()))
  f.close()

for i in range(100):
  file_name = "eval_data/example"+str(i)+".txt"
  with open(file_name,'w') as f:
    f.write(str(combine_cq[i]['context'].strip())+"|||"+str(combine_cq[i]['question'].strip()))
  f.close()

Cloning into 'squash-generation'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 375 (delta 1), reused 1 (delta 0), pack-reused 369
Receiving objects: 100% (375/375), 397.70 KiB | 577.00 KiB/s, done.
Resolving deltas: 100% (177/177), done.
rm: cannot remove 'train_data': No such file or directory
rm: cannot remove 'eval_data': No such file or directory


In [0]:
!pip install -r squash-generation/requirements.txt

     |████████████████████████████████| 102kB 10.3MB/s 
     |████████████████████████████████| 5.7MB 35.1MB/s 
     |████████████████████████████████| 1.7MB 56.9MB/s 
     |████████████████████████████████| 133kB 76.7MB/s 
     |████████████████████████████████| 5.6MB 26.8MB/s 
     |████████████████████████████████| 163kB 59.9MB/s 
     |████████████████████████████████| 552kB 53.0MB/s 
     |████████████████████████████████| 92kB 13.3MB/s 
     |████████████████████████████████| 51kB 8.8MB/s 
     |████████████████████████████████| 2.2MB 47.7MB/s 
     |████████████████████████████████| 2.8MB 64.3MB/s 
     |████████████████████████████████| 133kB 55.2MB/s 
     |████████████████████████████████| 286kB 58.4MB/s 
     |████████████████████████████████| 256kB 63.6MB/s 
     |████████████████████████████████| 61kB 11.0MB/s 
     |████████████████████████████████| 61kB 9.1MB/s 
     |████████████████████████████████| 92kB 14.2MB/s 
     |████████████████████████████████| 13.1MB 19.4MB/s

In [0]:
import os
import math
import logging
from pprint import pformat
from collections import defaultdict
from itertools import chain

import torch
from torch.nn.parallel import DistributedDataParallel
from torch.utils.data import DataLoader, TensorDataset
from ignite.engine import Engine, Events
from ignite.handlers import ModelCheckpoint
from ignite.metrics import Accuracy, Loss, MetricsLambda, RunningAverage
from ignite.contrib.handlers import ProgressBar, PiecewiseLinear
from ignite.contrib.handlers.tensorboard_logger import TensorboardLogger, OutputHandler, OptimizerParamsHandler
from pytorch_pretrained_bert import (OpenAIAdam, OpenAIGPTLMHeadModel, OpenAIGPTTokenizer,
                                     GPT2LMHeadModel, GPT2Tokenizer, WEIGHTS_NAME, CONFIG_NAME)
#from tqdm import tqdm_notebook

SPECIAL_TOKENS = [
    "<bos>", "<eos>", "<context>","<question>","<answer>","<pad>"
]
MODEL_INPUTS = ["input_ids", "lm_labels", "token_type_ids"]
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
bos, eos, context,question,answer,pad = \
          tokenizer.convert_tokens_to_ids(SPECIAL_TOKENS)
tokenizer.set_special_tokens(SPECIAL_TOKENS)

100%|██████████| 456318/456318 [00:01<00:00, 418264.84B/s]


In [0]:
#answer first 

from torch.utils.data import Dataset, DataLoader
def get_example(txt):
  inst = {}
  try:
    context,question = txt.strip().split("|||")
  except:
    print(txt)
  toks1 = tokenizer.tokenize(context.strip())
  toks2 = tokenizer.tokenize(question.strip())
 
  if (len(toks1) + len(toks2))< 1019:
    inst['context'] = tokenizer.convert_tokens_to_ids(toks1)
    inst['question'] = tokenizer.convert_tokens_to_ids(toks2)
  else:
    #count = count+1
    max_len_for_article = 1019 - len(toks2)
    inst['context'] = tokenizer.convert_tokens_to_ids(toks1)[:max_len_for_article]
    inst['question'] = tokenizer.convert_tokens_to_ids(toks2)
    
  bos, eos, context,question,answer,pad = \
          tokenizer.convert_tokens_to_ids(SPECIAL_TOKENS)
  instance = {}
  
  sequence = [bos] + inst['context'] 
  token_types = [context for _ in range(len(inst['context'])+1)]
  lm_labels = [-1 for _ in range(len(inst['context'])+1)]
  
  
  sequence.extend([question]+inst['question']+[eos])
  token_types.extend([question for _ in range(len(inst['question'])+2)])
  lm_labels.extend([question]+inst['question']+[eos])
  
  
  
  
  #instance["input_ids"] = torch.tensor(sequence)
  #instance["token_type_ids"] = torch.tensor(token_types)
  #instance["lm_labels"] = torch.tensor(lm_labels)
  return torch.tensor(sequence),torch.tensor(token_types), torch.tensor(lm_labels)
  

def pad_and_sort_batch(DataLoaderBatch):
  batch_size = len(DataLoaderBatch)
  batch_split = list(zip(*DataLoaderBatch))
  
  seqs,types,labels = batch_split[0],batch_split[1],batch_split[2]
  seqs = [seq.tolist() for seq in seqs]
  types = [typ.tolist() for typ in types] 
  labels = [lb.tolist() for lb in labels]
  
  max_len = max([len(seq) for seq in seqs])
  
  
  seqs = [seq+[pad]*(max_len-len(seq)) for seq in seqs]
  types = [seq+[pad]*(max_len-len(seq)) for seq in types] 
  labels = [seq+[pad]*(max_len-len(seq)) for seq in labels]
  
  return torch.tensor(seqs),torch.tensor(types),torch.tensor(labels)
  
  

class MyDataset(Dataset):
  def __init__(self,path):
    
    self.path = path
    self.data_files = os.listdir(path)

  def __getitem__(self, idx):
    with open(self.path+"/"+self.data_files[idx]) as f:
      txt = " ".join(f.readlines())
    return get_example(txt)

  def __len__(self):
      return len(self.data_files)
    
train_dataset = MyDataset("train_data")
eval_dataset = MyDataset("eval_data")
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=False, num_workers=4,collate_fn = pad_and_sort_batch)
eval_loader =  DataLoader(eval_dataset, batch_size=2, shuffle=False, num_workers=4,collate_fn = pad_and_sort_batch)

In [0]:
for step, (x, y,z) in tqdm_notebook(enumerate(train_loader)): 
  if x.size()[1]>1024  or y.size()[1]>1024 or z.size()[1]>1024:
    print(tokenizer.decode(x[0].tolist()))
    print(step)
    break

Token indices sequence length is longer than the specified maximum  sequence length for this OpenAI GPT model (1053 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this OpenAI GPT model (1406 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this OpenAI GPT model (1116 > 1024). Running this sequence through the model will result in indexing errors


KeyboardInterrupt: ignored

In [0]:
args = {'distributed':False,'train_batch_size':2,'valid_batch_size':2,
       "device":"cuda",
        "model_checkpoint":"gpt2",
        "lr":6.25e-5,
        "local_rank":-1,
        "n_epochs":15,
        "max_norm":1.0,
        "gradient_accumulation_steps":8,
        "output_dir":"/content/drive/My Drive/harvest/","fp16":""
        
       }
from collections import namedtuple
args = namedtuple("GenericDict",args.keys())(**args)
#torch.cuda.empty()
model_class = GPT2LMHeadModel if "gpt2" in args.model_checkpoint else OpenAIGPTLMHeadModel
model = model_class.from_pretrained("/content/drive/My Drive/harvest/")
tokenizer.set_special_tokens(SPECIAL_TOKENS)
model.set_num_special_tokens(len(SPECIAL_TOKENS))
model.to(args.device)
optimizer = OpenAIAdam(model.parameters(), lr=args.lr)
# Training function and trainer
def update(engine, batch):
    model.train()
    #batch = tuple(input_tensor.to(args.device) for input_tensor in batch)
    
    
    lm_loss = model(batch[0].to(args.device), token_type_ids=batch[1].to(args.device),
                    lm_labels=batch[2].to(args.device))
    loss = lm_loss / args.gradient_accumulation_steps
    if args.fp16:
        with amp.scale_loss(loss, optimizer) as scaled_loss:
            scaled_loss.backward()
        torch.nn.utils.clip_grad_norm_(amp.master_params(optimizer), args.max_norm)
    else:
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), args.max_norm)
    if engine.state.iteration % args.gradient_accumulation_steps == 0:
        optimizer.step()
        optimizer.zero_grad()
    return loss.item()
trainer = Engine(update)
# Evaluation function and evaluator (evaluator output is the input of the metrics)
def inference(engine, batch):
    model.eval()
    with torch.no_grad():
        #batch = tuple(input_tensor.to(args.device) for input_tensor in batch)
        #input_ids, lm_labels, token_type_ids = batch
        input_ids = batch[0].to(args.device)
        lm_labels = batch[1].to(args.device)
        token_type_ids = batch[2].to(args.device)

        # logger.info(tokenizer.decode(input_ids[0, :].tolist()))
        model_outputs = model(input_ids, token_type_ids=token_type_ids)
        lm_logits = model_outputs[0]

        lm_logits_flat_shifted = lm_logits[..., :-1, :].contiguous().view(-1, lm_logits.size(-1))
        lm_labels_flat_shifted = lm_labels[..., 1:].contiguous().view(-1)

        return lm_logits_flat_shifted, lm_labels_flat_shifted
evaluator = Engine(inference)
trainer.add_event_handler(Events.EPOCH_COMPLETED, lambda _: evaluator.run(eval_loader))
if args.n_epochs < 1:
    trainer.add_event_handler(Events.COMPLETED, lambda _: evaluator.run(eval_loader))

t_total value of -1 results in schedule not being applied


In [0]:
def average_distributed_scalar(scalar, args):
    """ Average a scalar over the nodes if we are in distributed training. We use this for distributed evaluation. """
    if args.local_rank == -1:
        return scalar
    scalar_t = torch.tensor(scalar, dtype=torch.float, device=args.device) / torch.distributed.get_world_size()
    torch.distributed.all_reduce(scalar_t, op=torch.distributed.ReduceOp.SUM)
    return scalar_t.item()
  
# Linearly decrease the learning rate from lr to zero
scheduler = PiecewiseLinear(optimizer, "lr", [(0, args.lr), (args.n_epochs * len(train_loader), 0.0)])
trainer.add_event_handler(Events.ITERATION_STARTED, scheduler)

# Prepare metrics - note how we compute distributed metrics
RunningAverage(output_transform=lambda x: x).attach(trainer, "loss")
metrics = {
    "nll": Loss(torch.nn.CrossEntropyLoss(ignore_index=-1))
}
metrics.update({
    "average_nll": MetricsLambda(average_distributed_scalar, metrics["nll"], args)
})
metrics["average_ppl"] = MetricsLambda(math.exp, metrics["average_nll"])
for name, metric in metrics.items():
    metric.attach(evaluator, name)

# On the main process: add progress bar, tensorboard, checkpoints and save model, configuration and tokenizer before we start to train
if args.local_rank in [-1, 0]:
    pbar = ProgressBar(persist=True)
    pbar.attach(trainer, metric_names=["loss"])
    evaluator.add_event_handler(Events.COMPLETED, lambda _: pbar.log_message("Validation: %s" % pformat(evaluator.state.metrics)))

    tb_logger = TensorboardLogger(log_dir=args.output_dir)
    tb_logger.attach(trainer, log_handler=OutputHandler(tag="training", metric_names=["loss"]), event_name=Events.ITERATION_COMPLETED)
    tb_logger.attach(trainer, log_handler=OptimizerParamsHandler(optimizer), event_name=Events.ITERATION_STARTED)
    tb_logger.attach(evaluator, log_handler=OutputHandler(tag="validation", metric_names=list(metrics.keys()), another_engine=trainer), event_name=Events.EPOCH_COMPLETED)

    checkpoint_handler = ModelCheckpoint("/content/drive/My Drive/harvest/", 'checkpoint', save_interval=10000, n_saved=5)
    trainer.add_event_handler(Events.ITERATION_COMPLETED, checkpoint_handler, {'mymodel': getattr(model, 'module', model)})  # "getattr" take care of distributed encapsulation

    #torch.save(args, tb_logger.writer.log_dir + '/model_training_args.bin')
    getattr(model, 'module', model).config.to_json_file(os.path.join(tb_logger.writer.log_dir, CONFIG_NAME))
    tokenizer.save_vocabulary(tb_logger.writer.log_dir)

# Run the training
trainer.run(train_loader, max_epochs=args.n_epochs)

# On the main process: close tensorboard logger and rename the last checkpoint (for easy re-loading with OpenAIGPTModel.from_pretrained method)
if args.local_rank in [-1, 0] and args.n_epochs > 0:
    os.rename(checkpoint_handler._saved[-1][1][-1], os.path.join(tb_logger.writer.log_dir, WEIGHTS_NAME))  # TODO: PR in ignite to have better access to saved file paths (cleaner)
    tb_logger.close()

Buffered data was truncated after reaching the output size limit.Buffered data was truncated after reaching the output size limit.

In [0]:
!rm -r logs